<a href="https://colab.research.google.com/github/Akshayextreme/Mood_detection_hackathon/blob/master/Mood_detector_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/aithon2020-level-2

/content/drive/My Drive/Colab Notebooks/aithon2020-level-2


In [ ]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import layers
import tensorflow as tf

# Data Preprocessing

In [ ]:
df = pd.read_csv('./data/aithon2020_level2_traning.csv')

In [ ]:
X = df.iloc[:,1:].values
y = df.emotion.values

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.2, random_state=43, shuffle=True)

In [ ]:
def VGG16_prep(train_data):
    rgb_list = []
    #Resizing image data from 48x48 to 224x224
    for i in range(train_data.shape[0]):
        image = Image.fromarray(np.uint8(train_data[i].reshape(48, 48))).convert('RGB')
        image = image.resize((224, 224), Image.BILINEAR)
        rgb_list.append(np.asarray(image))

    rgb_arr = np.stack([rgb_list],axis=4)
    rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)
    # rgb_arr_to_3d = np.expand_dims(rgb_arr_to_3d.astype('float32'), axis=0)
    return preprocess_input(rgb_arr_to_3d.astype('float32'))

In [ ]:
X_train = VGG16_prep(X_train)
X_test = VGG16_prep(X_test)

In [ ]:
X_train.shape

(8653, 224, 224, 3)

In [ ]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 3
print("Shape before one-hot encoding: ", y_train.shape)
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", y_train.shape)

Shape before one-hot encoding:  (8653,)
Shape after one-hot encoding:  (8653, 3)


In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        #rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()#rescale=1./255)

In [ ]:
batch_size = 64 # @param {type: "slider", min:1, max:128}
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

# EfficientNet as Feature Extractor

In [ ]:
IMG_SIZE = 224
NUM_CLASSES = 3
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    # x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    x = layers.Dense(4096, activation="relu", name="dense1")(x)
    x = layers.Dense(1028, activation="relu", name="dense2")(x)
    x = layers.Dropout(0.3, name="dense_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=NUM_CLASSES)

checkpoint_filepath = './checkpoint/extract_feature_Effnet.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
epochs = 25  # @param {type: "slider", min:8, max:80}
hist = model.fit(train_generator,
                 epochs=epochs, 
                 validation_data=test_generator, 
                 callbacks=[model_checkpoint_callback],
                 verbose=2)
plot_hist(hist)

# Fine-Tuning EfficientNet

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

In [ ]:
!rm -rf ./logs/train/*
!rm -rf ./logs/validation/*
checkpoint_filepath = './checkpoint/Effnet_finetune.h5'
callbacks = [
             tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
epochs = 30  # @param {type: "slider", min:8, max:80}
hist = model.fit(train_generator,
                 epochs=epochs, 
                 validation_data=test_generator, 
                 callbacks=callbacks,
                 verbose=2)
plot_hist(hist)